#rf_custom.py

In [1]:

# simple_random_forest.py
# Implementación sencilla de un Random Forest propio usando árboles de scikit-learn.
# Solo la clase con fit / predict (y predict_proba para clasificación).

from typing import Optional, Union
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import LabelEncoder


class SimpleRandomForest:
    """
    Random Forest minimalista (bagging de árboles) con interfaz estilo scikit-learn.

    - Usa bootstrap de ejemplos para cada árbol.
    - Usa submuestreo aleatorio de características por división a través de `max_features`
      (delegado al árbol base de scikit-learn).
    - Agregación:
        * Clasificación: promedio de probabilidades y argmax.
        * Regresión: promedio de predicciones.

    Parámetros
    ----------
    n_estimators : int, default=100
        Número de árboles en el ensamble.
    max_features : {'sqrt','log2','all'} o int o float en (0,1], default='sqrt'
        Número de características a considerar en cada división del árbol base.
        Se pasa directo al árbol (traducción: 'all' -> None).
    criterion : str, default='gini' (clasificación) o 'squared_error' (regresión)
        Criterio del árbol. Si task='auto', se ignora este valor y se elige por tipo.
    max_depth : int o None, default=None
        Profundidad máxima de cada árbol.
    random_state : int o None, default=None
        Semilla global para reproducibilidad.
    task : {'auto','classification','regression'}, default='auto'
        Tipo de problema. Si 'auto', se infiere a partir de y.

    Atributos (tras fit)
    --------------------
    estimators_ : list
        Lista de árboles entrenados.
    task_ : str
        Tipo inferido: 'classification' o 'regression'.
    classes_ : np.ndarray (solo clasificación)
        Clases en el orden usado para agregación.
    n_features_in_ : int
        Número de características de entrada.
    """

    def __init__(
        self,
        n_estimators: int = 100,
        max_features: Union[str, int, float] = "sqrt",
        criterion: Optional[str] = None,
        max_depth: Optional[int] = None,
        random_state: Optional[int] = None,
        task: str = "auto",
    ):
        self.n_estimators = int(n_estimators)
        self.max_features = max_features
        self.criterion = criterion
        self.max_depth = max_depth
        self.random_state = random_state
        self.task = task

        # Set en fit
        self.estimators_ = []
        self.task_ = None
        self.classes_ = None
        self._le = None
        self.n_features_in_ = None
        self._rng = None

    # --------------------------
    # Utilidades internas
    # --------------------------
    def _resolve_max_features(self, n_features: int):
        """Traduce el argumento max_features a lo que esperan los árboles de sklearn."""
        mf = self.max_features
        if mf == "all":
            return None  # sklearn: None => usa todas
        if isinstance(mf, str):
            if mf in {"sqrt", "log2"}:
                return mf
            raise ValueError("max_features string debe ser 'sqrt', 'log2' o 'all'.")
        if isinstance(mf, (int, np.integer)):
            if 1 <= mf <= n_features:
                return int(mf)
            raise ValueError("max_features int debe estar en [1, n_features].")
        if isinstance(mf, float):
            if 0.0 < mf <= 1.0:
                k = max(1, int(np.floor(mf * n_features)))
                return k
            raise ValueError("max_features float debe estar en (0, 1].")
        # None no es válido aquí (usar 'all' para todas)
        raise ValueError("max_features no válido.")

    def _bootstrap_sample(self, X: np.ndarray, y: np.ndarray, rng: np.random.RandomState):
        """Devuelve una muestra bootstrap de (X, y) del mismo tamaño que el conjunto original."""
        n_samples = X.shape[0]
        idx = rng.randint(0, n_samples, size=n_samples)  # con reemplazo
        return X[idx], y[idx]

    def _infer_task(self, y: np.ndarray) -> str:
        if self.task in ("classification", "regression"):
            return self.task
        y_type = type_of_target(y)
        if y_type in ("binary", "multiclass"):
            return "classification"
        elif y_type in ("continuous",):
            return "regression"
        else:
            # Heurística simple: si y es entera -> clasificación, si no -> regresión
            return "classification" if np.issubdtype(np.asarray(y).dtype, np.integer) else "regression"

    # --------------------------
    # API principal
    # --------------------------
    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X debe ser 2D (n_samples, n_features).")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X y y deben tener el mismo número de filas.")

        self.n_features_in_ = X.shape[1]
        self.task_ = self._infer_task(y)
        self._rng = np.random.RandomState(self.random_state)

        max_features = self._resolve_max_features(self.n_features_in_)

        self.estimators_ = []

        if self.task_ == "classification":
            # Aseguramos un orden global de clases para la agregación
            self._le = LabelEncoder().fit(y)
            self.classes_ = self._le.classes_

            # Criterio por defecto si no se especifica
            criterion = self.criterion if self.criterion is not None else "gini"

            for _ in range(self.n_estimators):
                Xi, yi = self._bootstrap_sample(X, y, self._rng)
                seed = int(self._rng.randint(0, 2**31 - 1))
                tree = DecisionTreeClassifier(
                    criterion=criterion,
                    max_depth=self.max_depth,
                    max_features=max_features,
                    random_state=seed,
                )
                tree.fit(Xi, yi)
                self.estimators_.append(tree)

        else:  # regresión
            criterion = self.criterion if self.criterion is not None else "squared_error"

            for _ in range(self.n_estimators):
                Xi, yi = self._bootstrap_sample(X, y, self._rng)
                seed = int(self._rng.randint(0, 2**31 - 1))
                tree = DecisionTreeRegressor(
                    criterion=criterion,
                    max_depth=self.max_depth,
                    max_features=max_features,
                    random_state=seed,
                )
                tree.fit(Xi, yi)
                self.estimators_.append(tree)

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        if not self.estimators_:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")

        X = np.asarray(X)

        if self.task_ == "classification":
            # Promedio de probabilidades con alineación de clases
            proba = self.predict_proba(X)
            # argmax sobre el eje de clases y deshacer codificación
            y_ind = np.argmax(proba, axis=1)
            return self.classes_[y_ind]
        else:
            # Promedio de predicciones de regresión
            preds = np.column_stack([est.predict(X) for est in self.estimators_])
            return preds.mean(axis=1)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """Probabilidades promedio para clasificación. No disponible para regresión."""
        if self.task_ != "classification":
            raise AttributeError("predict_proba solo está disponible para clasificación.")

        X = np.asarray(X)
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        proba_sum = np.zeros((n_samples, n_classes), dtype=float)

        for est in self.estimators_:
            # Probabilidades del árbol actual (puede faltar alguna clase en el bootstrap)
            est_proba = est.predict_proba(X)  # (n_samples, n_classes_est)
            est_classes = est.classes_         # clases vistas por este árbol
            # Mapear columnas a índice global de clases
            # Dado que LabelEncoder ordena y las clases_ del árbol también están ordenadas,
            # podemos ubicar con searchsorted.
            idx_map = np.searchsorted(self.classes_, est_classes)
            # Sumar en las columnas correspondientes
            proba_sum[:, idx_map] += est_proba

        # Promedio
        proba_avg = proba_sum / float(len(self.estimators_))

        # Normalización por si algún árbol omitió clases (para robustez numérica)
        row_sums = proba_avg.sum(axis=1, keepdims=True)
        # Evitar división por cero: si alguna fila suma 0 (muy raro), asignar uniforme
        zero_rows = (row_sums[:, 0] == 0.0)
        if np.any(zero_rows):
            proba_avg[zero_rows, :] = 1.0 / n_classes
            row_sums = proba_avg.sum(axis=1, keepdims=True)
        proba_avg /= row_sums

        return proba_avg

    # Compatibilidad mínima con API sklearn
    def get_params(self, deep: bool = True):
        return {
            "n_estimators": self.n_estimators,
            "max_features": self.max_features,
            "criterion": self.criterion,
            "max_depth": self.max_depth,
            "random_state": self.random_state,
            "task": self.task,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self


##rf_sklearn.py

In [2]:
# simple_random_forest_sklearn.py
# Implementación mínima de Random Forest usando directamente scikit-learn.
# Mantiene una API similar a la versión "propia": fit / predict / predict_proba.

from typing import Optional, Union
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.utils.multiclass import type_of_target


class SimpleRandomForestSK:
    """
    Envoltura ligera sobre RandomForest de scikit-learn con una interfaz simple y consistente.

    Parámetros
    ----------
    n_estimators : int, default=100
        Número de árboles.
    max_features : {'sqrt','log2','all'} o int o float en (0,1], default='sqrt'
        Subconjunto de características por división. 'all' -> usa todas (None en sklearn).
    criterion : str, default='gini' (clasificación) o 'squared_error' (regresión)
        Criterio del árbol base.
    max_depth : int o None, default=None
        Profundidad máxima de cada árbol.
    random_state : int o None, default=None
        Semilla de aleatoriedad.
    task : {'auto','classification','regression'}, default='auto'
        Tipo de problema; si 'auto', se infiere de y.

    Atributos (tras fit)
    --------------------
    estimator_ : RandomForestClassifier o RandomForestRegressor
        Modelo entrenado.
    task_ : str
        'classification' o 'regression'.
    classes_ : np.ndarray (solo clasificación)
        Clases del modelo.
    n_features_in_ : int
        Número de columnas de X usadas en el entrenamiento.
    """

    def __init__(
        self,
        n_estimators: int = 100,
        max_features: Union[str, int, float] = "sqrt",
        criterion: Optional[str] = None,
        max_depth: Optional[int] = None,
        random_state: Optional[int] = None,
        task: str = "auto",
    ):
        self.n_estimators = int(n_estimators)
        self.max_features = max_features
        self.criterion = criterion
        self.max_depth = max_depth
        self.random_state = random_state
        self.task = task

        self.estimator_ = None
        self.task_ = None
        self.classes_ = None
        self.n_features_in_ = None

    # --------------------------
    # Utilidades internas
    # --------------------------
    def _resolve_max_features(self, n_features: int):
        """Traduce 'all' a None y valida enteros/floats dentro de rango."""
        mf = self.max_features
        if mf == "all":
            return None
        if isinstance(mf, (int, np.integer)):
            if 1 <= mf <= n_features:
                return int(mf)
            raise ValueError("max_features int debe estar en [1, n_features].")
        if isinstance(mf, float):
            if 0.0 < mf <= 1.0:
                return float(mf)
            raise ValueError("max_features float debe estar en (0, 1].")
        if mf in (None, "sqrt", "log2"):
            return mf
        raise ValueError("max_features debe ser 'sqrt', 'log2', 'all', None, int o float en (0,1].")

    def _infer_task(self, y: np.ndarray) -> str:
        if self.task in ("classification", "regression"):
            return self.task
        y_type = type_of_target(y)
        if y_type in ("binary", "multiclass"):
            return "classification"
        elif y_type in ("continuous",):
            return "regression"
        # fallback heurístico
        return "classification" if np.issubdtype(np.asarray(y).dtype, np.integer) else "regression"

    # --------------------------
    # API principal
    # --------------------------
    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X debe ser 2D (n_samples, n_features).")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X y y deben tener el mismo número de filas.")

        self.n_features_in_ = X.shape[1]
        self.task_ = self._infer_task(y)
        max_features = self._resolve_max_features(self.n_features_in_)

        if self.task_ == "classification":
            criterion = self.criterion if self.criterion is not None else "gini"
            self.estimator_ = RandomForestClassifier(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                max_features=max_features,
                criterion=criterion,
                bootstrap=True,
                random_state=self.random_state,
                n_jobs=-1,
            )
        else:
            criterion = self.criterion if self.criterion is not None else "squared_error"
            self.estimator_ = RandomForestRegressor(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                max_features=max_features,
                criterion=criterion,
                bootstrap=True,
                random_state=self.random_state,
                n_jobs=-1,
            )

        self.estimator_.fit(X, y)

        if self.task_ == "classification":
            self.classes_ = self.estimator_.classes_

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        if self.estimator_ is None:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")
        X = np.asarray(X)
        return self.estimator_.predict(X)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        if self.estimator_ is None:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")
        if self.task_ != "classification":
            raise AttributeError("predict_proba solo está disponible para clasificación.")
        X = np.asarray(X)
        return self.estimator_.predict_proba(X)

    # Compatibilidad mínima con API sklearn
    def get_params(self, deep: bool = True):
        return {
            "n_estimators": self.n_estimators,
            "max_features": self.max_features,
            "criterion": self.criterion,
            "max_depth": self.max_depth,
            "random_state": self.random_state,
            "task": self.task,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self


# model.pkl


In [8]:
# train_srf_iris_standalone.py
# Entrena y evalúa un Random Forest propio (bagging de árboles) sobre iris_train_clean.csv
# Todo en un solo archivo: clase + entrenamiento.

from typing import Optional, Union
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


class SimpleRandomForest:
    """
    Random Forest minimalista (bagging de árboles) con interfaz estilo scikit-learn.

    Parámetros
    ----------
    n_estimators : int, default=100
    max_features : {'sqrt','log2','all'} o int o float en (0,1], default='sqrt'
    criterion : str, default='gini' (clasif) o 'squared_error' (regresión)
    max_depth : int o None
    random_state : int o None
    task : {'auto','classification','regression'}
    """
    def __init__(
        self,
        n_estimators: int = 100,
        max_features: Union[str, int, float] = "sqrt",
        criterion: Optional[str] = None,
        max_depth: Optional[int] = None,
        random_state: Optional[int] = None,
        task: str = "auto",
    ):
        self.n_estimators = int(n_estimators)
        self.max_features = max_features
        self.criterion = criterion
        self.max_depth = max_depth
        self.random_state = random_state
        self.task = task

        self.estimators_ = []
        self.task_ = None
        self.classes_ = None
        self._le = None
        self.n_features_in_ = None
        self._rng = None

    # -------- utilidades internas --------
    def _resolve_max_features(self, n_features: int):
        mf = self.max_features
        if mf == "all":
            return None  # sklearn usa None para "todas"
        if isinstance(mf, str):
            if mf in {"sqrt", "log2"}:
                return mf
            raise ValueError("max_features string debe ser 'sqrt', 'log2' o 'all'.")
        if isinstance(mf, (int, np.integer)):
            if 1 <= mf <= n_features:
                return int(mf)
            raise ValueError("max_features int debe estar en [1, n_features].")
        if isinstance(mf, float):
            if 0.0 < mf <= 1.0:
                return max(1, int(np.floor(mf * n_features)))
            raise ValueError("max_features float debe estar en (0, 1].")
        raise ValueError("max_features no válido.")

    def _bootstrap_sample(self, X: np.ndarray, y: np.ndarray, rng: np.random.RandomState):
        n_samples = X.shape[0]
        idx = rng.randint(0, n_samples, size=n_samples)  # con reemplazo
        return X[idx], y[idx]

    def _infer_task(self, y: np.ndarray) -> str:
        if self.task in ("classification", "regression"):
            return self.task
        y_type = type_of_target(y)
        if y_type in ("binary", "multiclass"):
            return "classification"
        elif y_type in ("continuous",):
            return "regression"
        return "classification" if np.issubdtype(np.asarray(y).dtype, np.integer) else "regression"

    # -------------- API --------------
    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X debe ser 2D (n_samples, n_features).")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X y y deben tener el mismo número de filas.")

        self.n_features_in_ = X.shape[1]
        self.task_ = self._infer_task(y)
        self._rng = np.random.RandomState(self.random_state)
        max_features = self._resolve_max_features(self.n_features_in_)
        self.estimators_ = []

        if self.task_ == "classification":
            self._le = LabelEncoder().fit(y)
            self.classes_ = self._le.classes_
            criterion = self.criterion if self.criterion is not None else "gini"
            for _ in range(self.n_estimators):
                Xi, yi = self._bootstrap_sample(X, y, self._rng)
                seed = int(self._rng.randint(0, 2**31 - 1))
                tree = DecisionTreeClassifier(
                    criterion=criterion,
                    max_depth=self.max_depth,
                    max_features=max_features,
                    random_state=seed,
                )
                tree.fit(Xi, yi)
                self.estimators_.append(tree)
        else:
            criterion = self.criterion if self.criterion is not None else "squared_error"
            for _ in range(self.n_estimators):
                Xi, yi = self._bootstrap_sample(X, y, self._rng)
                seed = int(self._rng.randint(0, 2**31 - 1))
                tree = DecisionTreeRegressor(
                    criterion=criterion,
                    max_depth=self.max_depth,
                    max_features=max_features,
                    random_state=seed,
                )
                tree.fit(Xi, yi)
                self.estimators_.append(tree)
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        if not self.estimators_:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")
        X = np.asarray(X)
        if self.task_ == "classification":
            proba = self.predict_proba(X)
            y_ind = np.argmax(proba, axis=1)
            return self.classes_[y_ind]
        preds = np.column_stack([est.predict(X) for est in self.estimators_])
        return preds.mean(axis=1)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        if self.task_ != "classification":
            raise AttributeError("predict_proba solo está disponible para clasificación.")
        X = np.asarray(X)
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        proba_sum = np.zeros((n_samples, n_classes), dtype=float)
        for est in self.estimators_:
            est_proba = est.predict_proba(X)
            est_classes = est.classes_
            idx_map = np.searchsorted(self.classes_, est_classes)
            proba_sum[:, idx_map] += est_proba
        proba_avg = proba_sum / float(len(self.estimators_))
        row_sums = proba_avg.sum(axis=1, keepdims=True)
        zero_rows = (row_sums[:, 0] == 0.0)
        if np.any(zero_rows):
            proba_avg[zero_rows, :] = 1.0 / n_classes
            row_sums = proba_avg.sum(axis=1, keepdims=True)
        proba_avg /= row_sums
        return proba_avg

    def get_params(self, deep: bool = True):
        return {
            "n_estimators": self.n_estimators,
            "max_features": self.max_features,
            "criterion": self.criterion,
            "max_depth": self.max_depth,
            "random_state": self.random_state,
            "task": self.task,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self


# ======= Entrenamiento simple en Iris limpio =======
if __name__ == "__main__":
    CSV_PATH = "iris_train_clean.csv"  # ajusta si tu ruta es distinta
    TARGET_COLUMN = "target"
    RNG_SEED = 42

    df = pd.read_csv(CSV_PATH)
    if TARGET_COLUMN not in df.columns:
        raise ValueError(f"No existe la columna objetivo '{TARGET_COLUMN}' en el CSV.")

    X = df.drop(columns=[TARGET_COLUMN]).values
    y = df[TARGET_COLUMN].values

    # Por si vienen como 0.0,1.0,2.0 -> int
    if y.dtype.kind == "f":
        import numpy as _np
        if _np.allclose(y, y.round()):
            y = y.round().astype(int)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.3, random_state=RNG_SEED, stratify=y
    )

    srf = SimpleRandomForest(
        n_estimators=200,
        max_features="sqrt",
        max_depth=None,
        random_state=RNG_SEED,
        task="classification",
    ).fit(X_tr, y_tr)

    y_pred = srf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    cm = confusion_matrix(y_te, y_pred, labels=srf.classes_)
    report = classification_report(y_te, y_pred, digits=3)

    print("=== SimpleRandomForest (propio) en Iris (clean) ===")
    print(f"Accuracy: {acc:.3f}")
    print("\nClases (orden interno):", srf.classes_)
    print("Matriz de confusión:\n", cm)
    print("\nReporte de clasificación:\n", report)


=== SimpleRandomForest (propio) en Iris (clean) ===
Accuracy: 0.889

Clases (orden interno): [0 1 2]
Matriz de confusión:
 [[12  0  0]
 [ 0 10  2]
 [ 0  2 10]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0      1.000     1.000     1.000        12
           1      0.833     0.833     0.833        12
           2      0.833     0.833     0.833        12

    accuracy                          0.889        36
   macro avg      0.889     0.889     0.889        36
weighted avg      0.889     0.889     0.889        36



In [12]:
import pickle

# Define the filename for the pickle file
model_filename = '../model/srf_propio_model.pkl' # Changed filename to be more specific

# Save the trained SimpleRandomForest model (propio) to a pickle file
with open(model_filename, 'wb') as f:
    pickle.dump(srf, f) # Corrected variable name from srf_cleaned to srf

print(f"SimpleRandomForest (propio) model saved to {model_filename}")


SimpleRandomForest (propio) model saved to ../model/srf_propio_model.pkl


# Comparación

In [11]:
# train_srf_sklearn_standalone.py
# Entrena y evalúa un Random Forest usando scikit-learn (wrapper SimpleRandomForestSK).
# Todo en un solo archivo para evitar problemas de import.

from typing import Optional, Union
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import type_of_target
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


class SimpleRandomForestSK:
    """
    Envoltura ligera sobre RandomForest de scikit-learn con una interfaz simple y consistente.

    Parámetros
    ----------
    n_estimators : int, default=100
    max_features : {'sqrt','log2','all'} o int o float en (0,1], default='sqrt'
        'all' -> usa todas (None en sklearn).
    criterion : str, default='gini' (clasificación) o 'squared_error' (regresión)
    max_depth : int o None, default=None
    random_state : int o None, default=None
    task : {'auto','classification','regression'}, default='auto'
    """

    def __init__(
        self,
        n_estimators: int = 100,
        max_features: Union[str, int, float] = "sqrt",
        criterion: Optional[str] = None,
        max_depth: Optional[int] = None,
        random_state: Optional[int] = None,
        task: str = "auto",
    ):
        self.n_estimators = int(n_estimators)
        self.max_features = max_features
        self.criterion = criterion
        self.max_depth = max_depth
        self.random_state = random_state
        self.task = task

        self.estimator_ = None
        self.task_ = None
        self.classes_ = None
        self.n_features_in_ = None

    # --------------------------
    # Utilidades internas
    # --------------------------
    def _resolve_max_features(self, n_features: int):
        """Traduce 'all' a None y valida enteros/floats dentro de rango."""
        mf = self.max_features
        if mf == "all":
            return None
        if isinstance(mf, (int, np.integer)):
            if 1 <= mf <= n_features:
                return int(mf)
            raise ValueError("max_features int debe estar en [1, n_features].")
        if isinstance(mf, float):
            if 0.0 < mf <= 1.0:
                return float(mf)
            raise ValueError("max_features float debe estar en (0, 1].")
        if mf in (None, "sqrt", "log2"):
            return mf
        raise ValueError("max_features debe ser 'sqrt', 'log2', 'all', None, int o float en (0,1].")

    def _infer_task(self, y: np.ndarray) -> str:
        if self.task in ("classification", "regression"):
            return self.task
        y_type = type_of_target(y)
        if y_type in ("binary", "multiclass"):
            return "classification"
        elif y_type in ("continuous",):
            return "regression"
        return "classification" if np.issubdtype(np.asarray(y).dtype, np.integer) else "regression"

    # --------------------------
    # API principal
    # --------------------------
    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X debe ser 2D (n_samples, n_features).")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X y y deben tener el mismo número de filas.")

        self.n_features_in_ = X.shape[1]
        self.task_ = self._infer_task(y)
        max_features = self._resolve_max_features(self.n_features_in_)

        if self.task_ == "classification":
            criterion = self.criterion if self.criterion is not None else "gini"
            self.estimator_ = RandomForestClassifier(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                max_features=max_features,
                criterion=criterion,
                bootstrap=True,
                random_state=self.random_state,
                n_jobs=-1,
            )
        else:
            criterion = self.criterion if self.criterion is not None else "squared_error"
            self.estimator_ = RandomForestRegressor(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                max_features=max_features,
                criterion=criterion,
                bootstrap=True,
                random_state=self.random_state,
                n_jobs=-1,
            )

        self.estimator_.fit(X, y)

        if self.task_ == "classification":
            self.classes_ = self.estimator_.classes_

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        if self.estimator_ is None:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")
        X = np.asarray(X)
        return self.estimator_.predict(X)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        if self.estimator_ is None:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")
        if self.task_ != "classification":
            raise AttributeError("predict_proba solo está disponible para clasificación.")
        X = np.asarray(X)
        return self.estimator_.predict_proba(X)

    # Compatibilidad mínima con API sklearn
    def get_params(self, deep: bool = True):
        return {
            "n_estimators": self.n_estimators,
            "max_features": self.max_features,
            "criterion": self.criterion,
            "max_depth": self.max_depth,
            "random_state": self.random_state,
            "task": self.task,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self


# ======= Entrenamiento/Evaluación en Iris limpio =======
if __name__ == "__main__":
    CSV_PATH = "iris_train_clean.csv"  # ajusta la ruta si es necesario
    TARGET_COLUMN = "target"
    RNG_SEED = 42

    df = pd.read_csv(CSV_PATH)
    if TARGET_COLUMN not in df.columns:
        raise ValueError(f"No existe la columna objetivo '{TARGET_COLUMN}' en el CSV.")

    X = df.drop(columns=[TARGET_COLUMN]).values
    y = df[TARGET_COLUMN].values

    # Si vienen etiquetas 0.0,1.0,2.0 como float, castear a int
    if y.dtype.kind == "f" and np.allclose(y, y.round()):
        y = y.round().astype(int)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.3, random_state=RNG_SEED, stratify=y
    )

    rf = SimpleRandomForestSK(
        n_estimators=200,
        max_features="sqrt",     # típico en clasificación
        max_depth=None,          # árboles profundos
        random_state=RNG_SEED,
        task="classification",
    ).fit(X_tr, y_tr)

    y_pred = rf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    cm = confusion_matrix(y_te, y_pred, labels=rf.classes_)
    report = classification_report(y_te, y_pred, digits=3)

    print("=== RandomForest (sklearn wrapper) en Iris (clean) ===")
    print(f"Accuracy: {acc:.3f}")
    print("\nClases (orden interno):", rf.classes_)
    print("Matriz de confusión:\n", cm)
    print("\nReporte de clasificación:\n", report)


=== RandomForest (sklearn wrapper) en Iris (clean) ===
Accuracy: 0.917

Clases (orden interno): [0 1 2]
Matriz de confusión:
 [[12  0  0]
 [ 0 10  2]
 [ 0  1 11]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0      1.000     1.000     1.000        12
           1      0.909     0.833     0.870        12
           2      0.846     0.917     0.880        12

    accuracy                          0.917        36
   macro avg      0.918     0.917     0.917        36
weighted avg      0.918     0.917     0.917        36

